## authentication 鉴权

https://blog.csdn.net/wang839305939/article/details/78713124/

[Django基于Token的身份验证](https://www.cnblogs.com/li1992/articles/9723056.html)

利用服务器端的session（会话）和浏览器端的cookie来实现前后端的认证，由于http请求时是无状态的，服务器正常情况下是不知道当前请求之前有没有来过，这个时候我们如果要记录状态，就需要在服务器端创建一个会话(seesion),将同一个客户端的请求都维护在各自得会会话中，每当请求到达服务器端的时候，先去查一下该客户端有没有在服务器端创建seesion，如果有则已经认证成功了，否则就没有认证。

sessionid

### JWT  JSON Web Tokens

JWT标准的Token有三个部分:
- header: 
- payload
- signature 相当于前两段的摘要,用来防止其他人来篡改Token中的信息

In [1]:
HEADER = {'typ': 'JWP', 'alg': 'default'}
KEY = 'CHEN_FENG_YAO'
SALT = 'www.lanou3g.com'
TIME_OUT = 30 * 60  # 30min

In [27]:
from django.core import signing
def encrypt(obj):
    """加密"""
    value = signing.dumps(obj, key=KEY, salt=SALT)
    value = signing.b64_encode(value.encode()).decode()
    return value


def decrypt(src):
    """解密"""
    src = signing.b64_decode(src.encode()).decode()
    raw = signing.loads(src, key=KEY, salt=SALT)
    # print(type(raw))
    return raw

In [31]:
#There are two components here, separated by a ':'. The first component is a
# URLsafe base64 encoded JSON of the object passed to dumps(). The second
# component is a base64 encoded hmac/SHA1 hash of "$first_component:$secret"
signing.dumps("ginger", key=KEY, salt=SALT)

'ImdpbmdlciI:1jEA0h:2mYw3TUeoc3eUb1X5ADeoYdLQH0'

In [10]:
import time
import hashlib
def create_token(username):
    """生成token信息"""
    # 1. 加密头信息
    header = encrypt(HEADER)
    # 2. 构造Payload
    payload = {"username": username, "iat": time.time()}   # iat：Issued at，发行时间
    payload = encrypt(payload)
    # 3. 生成签名
    md5 = hashlib.md5()
    md5.update(("%s.%s" % (header, payload)).encode())
    signature = md5.hexdigest()
    token = "%s.%s.%s" % (header, payload, signature)
    # 存储到缓存中
    # cache.set(username, token, TIME_OUT)  # 将username作为key 
    return token
token = create_token("ginger") # 三段string header，payload，signature

In [13]:
token

'ZXlKMGVYQWlPaUpLVjFBaUxDSmhiR2NpT2lKa1pXWmhkV3gwSW4wOjFqRTlDaTphN1RxMkRyejdLLWVXM0FYakRGOTh1SXlnNms.ZXlKMWMyVnlibUZ0WlNJNkltZHBibWRsY2lJc0ltbGhkQ0k2TVRVNE5EUXpPVGt5T0M0ek9EQTNNVGczZlE6MWpFOUNpOlBVSGhkT0NTYmtxTFRXTHBsTmlvTHJVckFnSQ.62b2b66afddd8947468b620ad0b56417'

In [11]:
def get_payload(token):
    payload = str(token).split('.')[1]
    payload = decrypt(payload)
    return payload
get_payload(token)

<class 'dict'>


{'username': 'ginger', 'iat': 1584439928.3807187}

In [12]:
def check_token(token):
    username = get_username(token)
    last_token = cache.get(username)
    if last_token:
        return last_token == token
    return False
